In [ ]:
# Modules for deep learning and other operations
import os
from math import floor, ceil
import numpy as np
import torch
from torchvision import transforms
import pytorch_lightning as pl
# Modules with networks
import sys
sys.path.append('../../')
from decoder.net import SDecoder
from N2V.models.lvae import N2V
from N2V.masks import RowMask, ColumnMask, CrossMask
from utils.datasets import TrainDatasetUnsupervised, PredictDatasetUNet
# Modules for loading and viewing images
import tifffile
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Load the noisy images
# These will be a list of PyTorch tensors with dimensions: [Channels, Height, Width]
top_dir = '../../data/Fluo-C2DL-MSC/low_snr'
files = os.listdir(top_dir)
low_snr = [tifffile.imread(os.path.join(top_dir, file)) for file in files]
low_snr = [torch.from_numpy(image) for image in low_snr]

In [ ]:
cumulative_sum = 0
cumulative_size = 0
for i in range(len(low_snr)):
    cumulative_sum += low_snr[i].sum()
    cumulative_size += low_snr[i].numel()

low_snr_mean = cumulative_sum / cumulative_size

cumulative_deviation = 0
for i in range(len(low_snr)):
    cumulative_deviation += ((low_snr[i] - low_snr_mean)**2).sum()

low_snr_std = (cumulative_deviation / cumulative_size)**0.5

In [ ]:
# Check
idx = np.random.randint(len(low_snr))
plt.imshow(low_snr[idx][0])
plt.show()

In [ ]:
# Create training, validation and prediction dataloaders
batch_size = 4
crop_size = 256
n_iters = max(low_snr[0].shape[-2], low_snr[0].shape[-1]) // crop_size
transform = transforms.RandomCrop(crop_size)

train_val_set = TrainDatasetUnsupervised(low_snr, n_iters=n_iters, transform=transform)
train_set, val_set = torch.utils.data.random_split(train_val_set, (floor(0.8*len(train_val_set)), ceil(0.2*len(train_val_set))))
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=False)

predict_set = [image[:, :512, :512] for image in low_snr]

predict_set = PredictDatasetUNet(predict_set)
predict_loader = torch.utils.data.DataLoader(predict_set, batch_size=batch_size, shuffle=False)

In [ ]:
# Create the masking function. This will apply a median mask to the random pixels in a cross shape
mask_func = CrossMask(n_masks=10, search_area=(5, 5)).forward

# The number of features the clean signal code will have, the signal decoder will need to know this
s_code_features = 32

# Create the s decoder. This will learn to decode the signal code into a signal estimate
s_decoder = SDecoder(colour_channels=1, code_features=s_code_features)

# Create the UNet
# Pass the masking function as an argument
n2v = N2V(colour_channels=1,
          data_mean=low_snr_mean,
          data_std=low_snr_std,
          img_shape=(crop_size, crop_size),
          s_decoder=s_decoder,
          n_filters=s_code_features,
          mask_func=mask_func)

In [ ]:
# Where training logs and the trained parameters will be saved
checkpoint_path = '../../checkpoints/Fluo-C2DL-MSC/N2V'

trainer = pl.Trainer(default_root_dir=checkpoint_path,
                     accelerator='gpu',
                     devices=1,
                     max_epochs=500,
                     log_every_n_steps=len(train_set) // batch_size)

In [ ]:
# Train and save model
trainer.fit(n2v, train_loader, val_loader)
trainer.save_checkpoint(os.path.join(checkpoint_path, 'final_params.ckpt'))

In [ ]:
# Load the trained model
# Note the new hyperparameters: the loss does not need to be calculated so <mode_pred> is set True. The image shape <img_shape> is the same this time but would need to be changed if we were to evaluate on images of a different size to the training ones.
n2v = N2V.load_from_checkpoint(os.path.join(checkpoint_path, 'final_params.ckpt'),
                               s_decoder=s_decoder,
                               img_shape=(512, 512),
                               mode_pred=True).eval()

# Evaluate the model on the test set
# The returned "predictions" will be a list with length equal to the number of noisy images divided by batch size.
# Each element of the list is a tensor of shape [batch_size, colours, height, width]
predictions = trainer.predict(n2v, predict_loader)

In [ ]:
# Select a random image, its corresponding clean estimates and its ground truth
batch_idx = np.random.randint(len(predictions))
img_idx = np.random.randint(batch_size)

noisy_image = predict_set[batch_idx * batch_size + img_idx][0]
denoised = predictions[batch_idx][img_idx, 0]

vmin = np.percentile(noisy_image, 1)
vmax = np.percentile(noisy_image, 99.99)

# Display all three
fig, ax = plt.subplots(1, 2, figsize=[15, 15])
ax[0].imshow(noisy_image, cmap='inferno', vmin=vmin, vmax=vmax)
ax[0].axis('off')
ax[0].set_title('Input')

ax[1].imshow(denoised, cmap='inferno', vmin=vmin, vmax=vmax)
ax[1].axis('off')
ax[1].set_title('Denoised')

fig.show()